### Env checking

In [4]:
import torch
print(torch.__version__)                   # In ra version, ví dụ 2.5.1
print(torch.cuda.is_available())           # True nếu GPU dùng được
print(torch.cuda.get_device_name(0))       # Tên GPU nếu có


2.5.1
True
NVIDIA GeForce RTX 4050 Laptop GPU


In [5]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))
for gpu in gpus:
    print("GPU Name:", gpu.name)


TensorFlow version: 2.10.0
Num GPUs Available: 1
GPU Name: /physical_device:GPU:0


### YOLO Detecting

In [2]:
from ultralytics import YOLO
import cv2
import os
from typing import Union, List

def detect_license_plates(image_paths: Union[str, List[str]],
                          model_path,
                          save_dir=r"D:\ViTraLP\output_yolo_crop_test",  # Thư mục cố định
                          conf_thresh=0.4):
    os.makedirs(save_dir, exist_ok=True)

    # Load model
    model = YOLO(model_path)

    # Đảm bảo image_paths là list
    if isinstance(image_paths, str):
        image_paths = [image_paths]

    all_cropped_paths = []

    for image_path in image_paths:
        # Run inference
        results = model(image_path, conf=conf_thresh)
        result = results[0]
        boxes = result.boxes.xyxy.cpu().numpy()

        # Load ảnh gốc
        image = cv2.imread(image_path)
        cropped_paths = []

        for i, (x1, y1, x2, y2) in enumerate(boxes):
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            crop = image[y1:y2, x1:x2]

            basename = os.path.splitext(os.path.basename(image_path))[0]
            crop_filename = f"{basename}_plate_{i}.jpg"
            crop_path = os.path.join(save_dir, crop_filename)

            cv2.imwrite(crop_path, crop)
            cropped_paths.append(crop_path)

        all_cropped_paths.append({"image": image_path, "crops": cropped_paths})

    return all_cropped_paths


if __name__ == "__main__":
    image_paths = [
        r"C:\Users\ADMIN\Desktop\test_real\enhanced\1.jpg"
    ]
    model_path = "yolo_finetuned_weights/yolo11_rainy_real_200_best.pt"
    results = detect_license_plates(image_paths, model_path)

    for result in results:
        print(f"Cropped plates for {result['image']}:")
        for path in result['crops']:
            print(" -", path)



image 1/1 C:\Users\ADMIN\Desktop\test_real\enhanced\1.jpg: 480x640 2 LPs, 22.2ms
Speed: 2.5ms preprocess, 22.2ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Cropped plates for C:\Users\ADMIN\Desktop\test_real\enhanced\1.jpg:
 - D:\ViTraLP\output_yolo_crop_test\1_plate_0.jpg
 - D:\ViTraLP\output_yolo_crop_test\1_plate_1.jpg


### OCR recognizing

In [8]:
import os
from paddleocr import PaddleOCR

# Ép PaddleOCR chỉ dùng CPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Khởi tạo OCR với model nhận dạng tùy chỉnh
ocr_model = PaddleOCR(
    use_angle_cls=False,
    det=False,
    rec=True,
    text_recognition_model_dir=r"D:/ViTraLP/original_weights_paddle",
    lang='en',
    show_log=False
)

# Sửa 2 ký tự đầu tiên nếu nhận nhầm
def correct_first_two_digits(text):
    char_to_digit = {
        'O': '0', 'o': '0', 'Q': '0',
        'I': '1', 'l': '1', 'L': '1',
        'Z': '2', 'S': '5', 'B': '8', 'G': '6'
    }
    text = text.strip()
    if len(text) < 2:
        return text
    new_text = list(text)
    for i in range(2):
        if not new_text[i].isdigit():
            new_text[i] = char_to_digit.get(new_text[i], '0')
    return ''.join(new_text)

# Hàm OCR một ảnh và in ra từng kết quả
def recognize_and_print(img_path):
    result = ocr_model.ocr(img_path, cls=False)
    print(f"\n=== {os.path.basename(img_path)} ===")
    if result and result[0]:
        for line in result[0]:
            (box, (text, confidence)) = line
            corrected_text = correct_first_two_digits(text)
            print(f"Text: {corrected_text}, Confidence: {confidence:.2f}")
    else:
        print("No text detected")

# OCR toàn bộ ảnh trong thư mục
image_folder = r"D:\ViTraLP\output_yolo_crop_test"
valid_exts = ['.jpg', '.jpeg', '.png', '.bmp']

for filename in os.listdir(image_folder):
    ext = os.path.splitext(filename)[-1].lower()
    if ext in valid_exts:
        img_path = os.path.join(image_folder, filename)
        try:
            recognize_and_print(img_path)
        except Exception as e:
            print(f"{filename}: Error - {str(e)}")



=== 1_plate_0.jpg ===
Text: 51G100.96, Confidence: 0.94

=== 1_plate_1.jpg ===
Text: 100.96, Confidence: 0.99


In [7]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=False)

# In toàn bộ thuộc tính để tìm path chứa model
print(dir(ocr))

[2025/07/21 10:23:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_leng